# Import data

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
fpath = r'C:\Users\Mei\Instacart Basket Analysis\02 Data\Prepared data'

In [3]:
df = pd.read_pickle(os.path.join(fpath,'Entire_updated_merged.pkl'))

In [4]:
df_sub = df[:1000000]

# 1. Comparison of mean 

### The mean of the subset

In [5]:
df_sub.groupby(['department_id']).agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.577493
2,17.320781
3,16.084944
4,17.530458
5,14.763075
6,16.658449
7,17.031590
8,15.076662


### The mean of the entire dataset

In [6]:
df.groupby(['department_id']).agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.213779
6,16.439806
7,17.225802
8,15.340650


### The descriptive analysis between subset and the entire dataset

In [9]:
df_sub['order_number'].describe()

count    1000000.000000
mean          16.818548
std           17.433229
min            1.000000
25%            4.000000
50%           11.000000
75%           23.000000
max           99.000000
Name: order_number, dtype: float64

In [10]:
df['order_number'].describe()

count    3.240604e+07
mean     1.714223e+01
std      1.753527e+01
min      1.000000e+00
25%      5.000000e+00
50%      1.100000e+01
75%      2.400000e+01
max      9.900000e+01
Name: order_number, dtype: float64

### Conclusion: 
The outputs between subset with 1 million rows and the entire dataset don't have too much difference. 

The main reason is that the subset is a sample whose features can represent the entire data: From the Descriptive Analyses one can see that except the 'count' is differnt, other values between them are very close.

# 2. Creating the 'loyal flag' for the entire dataset 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32406041 entries, 0 to 32406040
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   product_id              int64  
 1   product_name            object 
 2   aisle_id                int64  
 3   department_id           int64  
 4   prices                  float64
 5   order_id                int64  
 6   user_id                 int64  
 7   eval_setting            object 
 8   order_number            int64  
 9   order_day_of_week       int64  
 10  order_hour_of_day       int64  
 11  days_since_prior_order  float64
 12  add_to_cart_order       int64  
 13  reordered               int64  
 14  price_range             object 
 15  busy days               object 
 16  busiest_period_of_day   object 
dtypes: float64(2), int64(10), object(5)
memory usage: 4.3+ GB


In [12]:
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [14]:
df['max_order'].value_counts()

99    1171363
8      811893
6      811423
9      810243
7      804002
       ...   
97      44949
98      44587
96      40453
2           6
1           5
Name: max_order, Length: 99, dtype: int64

In [15]:
df['max_order'].describe()

count    3.240604e+07
mean     3.305203e+01
std      2.515518e+01
min      1.000000e+00
25%      1.300000e+01
50%      2.600000e+01
75%      4.700000e+01
max      9.900000e+01
Name: max_order, dtype: float64

In [20]:
df.loc[df['max_order'] >47, 'loyalty_flag'] = 'loyal customer'
df.loc[df['max_order'] <= 13, 'loyalty_flag'] = 'new customer'
df.loc[(df['max_order'] > 13) & (df['max_order'] <= 47), 'loyalty_flag'] = 'regular customer'

In [21]:
df['loyalty_flag'].value_counts()

regular customer    15941976
new customer         8494733
loyal customer       7969332
Name: loyalty_flag, dtype: int64

# 3. Comparion among the three types of customers

##  Descriptive statistics 

In [35]:
df.groupby('loyalty_flag').agg({'prices':['mean','median','min','max','std']})

prices                                 
                       mean median  min      max         std
loyalty_flag                                                
loyal customer    10.090709    7.4  1.0  99999.0  300.216701
new customer      13.479703    7.4  1.0  99999.0  608.315461
regular customer  12.125410    7.4  1.0  99999.0  507.086129

In [36]:
df['loyalty_flag'].value_counts()

regular customer    15941976
new customer         8494733
loyal customer       7969332
Name: loyalty_flag, dtype: int64

## Conlusion: 
1) The minimum, maximum and median of the prices for these three groups are the same.

2) For each group, the distribution of prices is right skewed: the distribution of prices for new customer is the most right skewed, whereas the distribution for loyal customer is the least right skewed. 

2) The prices of the products by average bought by group of new customer is bigger than other groups; the standard deviation of the price for group of new customer is the largest, whereas regular customer's standard deviation of price is the smallest. 

# 4. Creating a 'spending flag'

In [38]:
df['mean_price'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [39]:
df.loc[df['mean_price']< 10, 'spender_flag'] = 'low spender'
df.loc[df['mean_price'] >= 10, 'spender_flag'] = 'high spender'

In [40]:
df['spender_flag'].value_counts()

low spender     31771789
high spender      634252
Name: spender_flag, dtype: int64

# 5. Creating the 'order frequency flag'

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32406041 entries, 0 to 32406040
Data columns (total 21 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   product_id              int64  
 1   product_name            object 
 2   aisle_id                int64  
 3   department_id           int64  
 4   prices                  float64
 5   order_id                int64  
 6   user_id                 int64  
 7   eval_setting            object 
 8   order_number            int64  
 9   order_day_of_week       int64  
 10  order_hour_of_day       int64  
 11  days_since_prior_order  float64
 12  add_to_cart_order       int64  
 13  reordered               int64  
 14  price_range             object 
 15  busy days               object 
 16  busiest_period_of_day   object 
 17  max_order               int64  
 18  loyalty_flag            object 
 19  mean_price              float64
 20  spender_flag            object 
dtypes: float64(3), int64(11), obj

In [42]:
df['median_day'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [44]:
df.loc[df['median_day'] <= 10, 'order_frequency'] = 'frequent customer'
df.loc[(df['median_day'] > 10) & (df['median_day']<= 20), 'order_frequency'] = 'regular customer'
df.loc[df['median_day'] >20, 'order_frequency'] = 'non-frequent customer'

In [45]:
df['order_frequency'].value_counts()

frequent customer        22790800
regular customer          6928407
non-frequent customer     2686834
Name: order_frequency, dtype: int64

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32406041 entries, 0 to 32406040
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   product_id              int64  
 1   product_name            object 
 2   aisle_id                int64  
 3   department_id           int64  
 4   prices                  float64
 5   order_id                int64  
 6   user_id                 int64  
 7   eval_setting            object 
 8   order_number            int64  
 9   order_day_of_week       int64  
 10  order_hour_of_day       int64  
 11  days_since_prior_order  float64
 12  add_to_cart_order       int64  
 13  reordered               int64  
 14  price_range             object 
 15  busy days               object 
 16  busiest_period_of_day   object 
 17  max_order               int64  
 18  loyalty_flag            object 
 19  mean_price              float64
 20  spender_flag            object 
 21  median_day              float

### I would like to delete the columns which have been introduced as intermediate variables, like 'max_order','mean_price' and 'median_day' in case some confusion will be caused. Am I right? (In this exercise I will delete them as the file is really big.)

In [47]:
df = df.drop(columns = ['max_order','mean_price','median_day'])

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32406041 entries, 0 to 32406040
Data columns (total 20 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   product_id              int64  
 1   product_name            object 
 2   aisle_id                int64  
 3   department_id           int64  
 4   prices                  float64
 5   order_id                int64  
 6   user_id                 int64  
 7   eval_setting            object 
 8   order_number            int64  
 9   order_day_of_week       int64  
 10  order_hour_of_day       int64  
 11  days_since_prior_order  float64
 12  add_to_cart_order       int64  
 13  reordered               int64  
 14  price_range             object 
 15  busy days               object 
 16  busiest_period_of_day   object 
 17  loyalty_flag            object 
 18  spender_flag            object 
 19  order_frequency         object 
dtypes: float64(2), int64(10), object(8)
memory usage: 5.1+ GB


In [49]:
df.to_pickle(os.path.join(fpath,'Entire_updated_merged_upd1.pkl'))